In [8]:
import numpy as np

In [9]:
def matrix2pgm(image_matrix,initial,output_image_file_name = str(np.random.randint(1,100000000000)) + '_default.pgm'):    
    with open(output_image_file_name, 'w') as file:
        file.writelines("% s\n" % pixel for pixel in initial)
        file.writelines("% s\n" % int(abs(pixel)) for pixel in image_matrix.flatten())

In [10]:
# Read a PGM Image and return image matrix and its initials separately
def read_pgm(image_file):
    with open(image_file, 'r') as file:
        image = file.read().splitlines()
    for pixel in range(3,len(image)):
        image[pixel] = int(image[pixel])
    image_matrix = np.array(image[4:]).reshape(int(image[2].split()[1]),int(image[2].split()[0]))
    return (image_matrix, image[:4])

In [11]:
def reduce(x):
    for i in range(len(x)):
        try:
            x1 = x.copy()
            for j in range(i+1,len(x)):
                try:
                    if not x[i].isdisjoint(x[j]):
                        x[i] = x[i].union(x[j])
                        x.remove(x[j])
                except: pass
            while len(x)<len(x1):
                x1 = x.copy()
                for j in range(i+1,len(x)):
                    try:
                        if not x[i].isdisjoint(x[j]):
                            x[i] = x[i].union(x[j])
                            x.remove(x[j])
                    except: pass
        except: pass
    return x

In [12]:
def label_matrix(image_matrix,V):
    label = dict()
    c = 0
    for i in range(image_matrix.shape[0]):
        for j in range(image_matrix.shape[1]):
            if image_matrix[i,j] in V:
                c = c+1
                label[(i,j)] = {c}
                for k in range(i-1,i+1+1):
                    for l in range(j-1,j+1+1):
                        try:
                            label[(i,j)] = label[(i,j)].union(label[(k,l)])
                        except: pass
                if len(label[(i,j)]) > 1:
                    label[(i,j)].remove(c)
    return label

In [13]:
def connected_components(image_file,V,return_image = True,backgroud_intencity = 0):
    image_matrix = read_pgm(image_file)
    d = label_matrix(image_matrix[0],V)
    x = list(d.values())
    y = reduce(x.copy())
    for f in range(len(y)):
        image_matrix1 = image_matrix[0].copy()
        for i in range(image_matrix1.shape[0]):
            for j in range(image_matrix1.shape[1]):
                try:
                    if next(iter(d[(i, j)])) in y[f]:
                        pass
                    else: image_matrix1[i,j] = backgroud_intencity
                except: image_matrix1[i,j] = backgroud_intencity
        if return_image:
            matrix2pgm(image_matrix1,image_matrix[1],output_image_file_name = str(f)+'-th object'+image_file+'.pgm')
    print('Total',len(y),'Objects')
    return(d,y)


In [14]:
z = connected_components('l.pgm',list(range(251)),return_image = True,backgroud_intencity = 0)

Total 6 Objects
